In [1]:
import chipwhisperer as cw
import logging
import time
import numpy as np
import random

In [2]:
PLATFORM = "CWLITEARM"

Let's build and flash the firmware:

In [ ]:
!cd ../hardware/victims/firmware/simpleserial-ecdsa && ./build.sh 1

In [ ]:
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()
cw.program_target(scope, cw.programmers.STM32FProgrammer, "../hardware/victims/firmware/simpleserial-ecdsa/simpleserial-ecdsa-CWLITEARM.hex")

Define some required functions:

In [5]:
def as_bytes(n, length):
    return bytearray(int.to_bytes(n, length=length, byteorder='big'))

def from_bytes(n):
    return int.from_bytes(n, byteorder='big')

# Correctness demonstration

In this section we will show that our cryptographic implementation provides correct results.

We will demonstrate this by checking whether the public key (based on the given private key) generated by our implementation matches the expected public key.

Then we will sign a message and using the `ecpy` library test whether or not our signature matches the public key.

First we will set the private key we want to use, and the expected public key we will obtain:

In [6]:
priv_key = 0x48775362b141bc1036fade0fe5b8d5b640ac23fe1608f81a13531e85fdf06ccf
priv_key_bytes = as_bytes(priv_key, 32)

expected_pub = [
    0xff67f157ba51a2543a007f8967abdae2e25fb4698439433b978539ce703efbf9, # X
    0x540256bd83026e3bcd0c1a466a942ca52e0d038f7b9394b169c5b3508618b46d  # Y
]

Now we will write to `k`, which computes the public key:

In [ ]:
print('setting k')
target.simpleserial_write('k', priv_key_bytes)

print('set k, wait')
time.sleep(10)

public_key_bytes = target.simpleserial_read('r', 64, ack=False)
public_key_bytes

In [ ]:
actual_pub_x = from_bytes(public_key_bytes[:32])
actual_pub_y = from_bytes(public_key_bytes[32:])
print(hex(actual_pub_x))
print(hex(actual_pub_y))

assert actual_pub_x == expected_pub[0], "Pub x was wrong"
assert actual_pub_y == expected_pub[1], "Pub y was wrong"

The `assert`s passed, the computed public key matches the expected public key.

Now let's sign a message!
We will sign the message `[1] * 64`.

In [ ]:
msg = [1] * 64
print('signing msg') 
target.simpleserial_write('z', bytearray(msg))
print('sign msg, wait') 
time.sleep(10)
print('sign msg, done') 

Let's read the result:

In [ ]:
signature = target.simpleserial_read('r', 64, ack=False)
print('got signature:', signature)

And check whether or not it is valid using `ecpy`:

In [ ]:
from ecpy.curves import Curve, Point
from ecpy.keys import ECPublicKey
from ecpy.ecdsa import ECDSA

curve = Curve.get_curve('secp256r1')

public_key = ECPublicKey(Point(actual_pub_x, actual_pub_y, curve))

ecdsa = ECDSA('RAW')
is_valid = ecdsa.verify(msg, signature, public_key)
if is_valid:
    print('Signature was valid')
else:
    print('Signature was NOT valid')

# Vulnerability 1 – Key-dependent branch

Set up the ADC:

In [12]:
scope.adc.samples = 3000

Define the functions required:

In [13]:
def get_trace(priv_key):
    print("tracing {0:b}".format(priv_key))
    priv_key_bytes = as_bytes(priv_key, 32)
    
    time.sleep(2)

    scope.arm()
    
    print('running f')
    target.simpleserial_write('f', priv_key_bytes)
    print('set f, wait')
    time.sleep(10)
    print('done wait f')
    
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')
    
    return scope.get_last_trace()

In [14]:
def trace_avg(key, n):
    xs = np.array([ get_trace(key) for _ in range(n) ])
    return np.mean(xs, axis=0)

Get measurements for the baseline:

In [ ]:
all_zero = trace_avg(0x0000000000000000000000000000000000000000000000000000000000000000, 5)
all_one  = trace_avg(0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff, 5)

Calculate the baseline:

In [16]:
def norm(xs):
    return (xs - np.min(xs)) / (np.max(xs) - np.min(xs))

In [ ]:
baseline = norm(all_one - all_zero)
cw.plot(baseline)

Execute the attack using three random scalars:

In [ ]:
figs = [cw.plot(), cw.plot(), cw.plot()]
traces = []

for i in range(3):
    priv_key = random.randint(0, 0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff)
    traces.append(get_trace(priv_key) * baseline)
    figs[i] *= cw.plot(traces[-1])

Key 1:

In [ ]:
figs[0]

Key 2:

In [ ]:
figs[1]

Key 3:

In [ ]:
figs[2]

Combined plot:

In [ ]:
fig = cw.plot()
for trace in traces:
    fig *= cw.plot(trace)
fig